# Deploy an image classification model to the Triton Inference Server on local compute

This notebook shows you how to deploy a Densenet image classification model to the high-performance [Triton Inference Server](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/index.html) on local compute.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [ ]:
import git
import os
import tempfile
import urllib
from azure.storage.blob import BlobClient
from pathlib import Path


url = "https://aka.ms/aml-triton-sample-model"
response = urllib.request.urlopen(url)

blob_client = BlobClient.from_blob_url(response.url)

# Download the model
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)


folder_path = prefix.joinpath("models", "triton", "densenet_onnx", "1")
model_file_path = prefix.joinpath(folder_path, "model.onnx")

# Save the model if it does not already exist
if not os.path.exists(model_file_path):
    os.makedirs(folder_path, exist_ok=True)
    with open(model_file_path, "wb") as my_blob:
        download_stream = blob_client.download_blob()
        my_blob.write(download_stream.readall())

## Register model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.

In [ ]:
from azureml.core.model import Model

model_path = prefix.joinpath("models")

model = Model.register(
    model_path=model_path,
    model_name="densenet_onnx",
    tags={"area": "Image classification", "type": "classification"},
    description="Image classification trained on Imagenet Dataset",
    workspace=ws,
)

print(model.name, model.description, model.version)

## Deploy webservice

In this case we deploy to the local compute, but for other options, see [our documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where?tabs=azcli)


In [ ]:
%%time

from azureml.core.webservice import LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig


service_name = "triton-densenet-onnx-local"
env = Environment.get(ws, "AzureML-Triton").clone("My-Triton")

for pip_package in ["pillow"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(
    # This entry script is where we dispatch a call to the Triton server
    entry_script="score_densenet.py",
    source_directory=prefix.joinpath("code", "deployment", "triton"),
    environment=env,
)

config = LocalWebservice.deploy_configuration(port=6789)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=config,
    overwrite=True,
)

service.wait_for_deployment(True)

## Test the webservice

In [ ]:
import requests

headers = {"Content-Type": "application/octet-stream"}

data_file = prefix.joinpath("data", "raw", "images", "peacock.jpg")
test_sample = open(data_file, "rb").read()
resp = requests.post(service.scoring_uri, data=test_sample, headers=headers)
print(resp.text)

## Delete the webservice and the downloaded model

In [ ]:
service.delete()
os.remove(model_file_path)

# Next steps

Try changing the deployment configuration to [deploy to Azure Kubernetes Service](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-azure-kubernetes-service?tabs=python) for higher availability and better scale.